In [31]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import os
import re

my_options = webdriver.ChromeOptions()

#最大化視窗
my_options.add_argument("--start-maximized")
#開啟無痕模式    
my_options.add_argument("--incognito")
#禁用彈出攔截
my_options.add_argument("--disable-popup-blocking")
#取消通知
my_options.add_argument("--disable-notifications")

driver = webdriver.Chrome(options=my_options)

In [32]:
base_url = "https://www.gutenberg.org"
language_page = f"{base_url}/browse/languages/zh"

folderPath = 'Gutenberg_ChineseBooks_v3.0'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

def openChrome():
    driver.get(language_page)

def contains_chinese(text):
    chinese_regex = re.compile(r'[\u4e00-\u9fff]')
    return bool(chinese_regex.search(text))

def getBookList():
    books = driver.find_elements(
        By.CSS_SELECTOR,'li.pgdbetext a'
    )

    booksList = []
    for i in books:
        bookTitle = i.get_attribute('innerText')
        if contains_chinese(bookTitle) :

            linkID = i.get_attribute('href').split('/')[-1]

            booksList.append(
                {
                    'title' : bookTitle,
                    'link' : i.get_attribute('href'),
                    'id' : linkID
                }
            )
    
    return booksList

titleList = []
def openPageDownload(bookList):
    for book in bookList:
        # open text link
        driver.get(f'https://www.gutenberg.org/cache/epub/{book['id']}/pg{book['id']}.txt')

        # create file and rename title
        specialText = r'[<>:\"/\\|?*\—\-=+\~`\']+|\s+'
        file_title = re.sub(specialText, "_", book['title'])

        # 若檔名相同，給與一個值附加在檔名
        while file_title in titleList:
            if '#' not in file_title:
                file_title = file_title + f"#1"
            else:
                booktitle = file_title.split('#')
                repeatNum = int(booktitle[1]) + 1
                file_title = booktitle[0] + f'#{repeatNum}'

        titleList.append(file_title)
        file_path = os.path.join(folderPath, f"{file_title}.txt")

        # get ineerText
        Text = driver.find_element(By.CSS_SELECTOR,'pre').get_attribute('innerText')
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(Text)
            
        chineseText = []
        onlyChineseRegex = r"[^\S\r\n]*[\u4e00-\u9fa5]+[^\S\r\n]*"
        with open(file_path,'r',encoding='utf-8') as f:
            for i in f.readlines():
                if re.match(onlyChineseRegex,i):
                    chineseText.append(i)

        with open(file_path, 'w', encoding='utf-8') as f:
            for text in chineseText:
                f.write(text)



def test():
    driver.get(f'https://www.gutenberg.org/cache/epub/25328/pg25328.txt')


In [33]:
openChrome()
AllBooks = getBookList()

print(f'包含中文書名共計 : {len(AllBooks)} 本')

openPageDownload(AllBooks)

包含中文書名共計 : 478 本
